Below are parameters for the Papercraft verifiable delay function (VDF).

In [2]:
from rok_estimator import *
from lattice_lib import *
from lattice_lib.util import *

In [6]:
mode = "C.L"

secpar = 128
kappa = 80
log_q = 62
rep = 57
t = 64 * 22
T = t * rep
n_vdf = 14 # Candidate module rank for VDF. Try bigger ones if the code complains. It should only depend on the conductor f of the cyclotomic ring and log_q. 

outer_ring_params = {
    "f": 24,
    "log_beta_sis_2": 44,
    "log_q": log_q,
    "secpar_target": secpar,
    "kappa_target": kappa
}
outer_ring = RingParam(**outer_ring_params)
inner_log_beta_sis_2 = bound_log_canon_2_from_log_coeff_inf(outer_ring, 0, dim=n_vdf * log_q)
inner_ring_params = {
    "f": 12,
    "log_beta_sis_2": inner_log_beta_sis_2 + 1,
    "log_q": 62,
}
inner_ring = RingParam(**inner_ring_params) # We actually care about a sub-ring of outer_ring of degree 12, but not another cyclotomic ring. Here we are only abusing the function to compute the SIS module rank. 
assert n_vdf >= inner_ring.n_sis

rel_params = {
    "wdim": t * n_vdf * log_q,
    "rep": rep,
    "n_rel" : 2,
    "log_beta_wit_inf": 0,
}

ops_params = {
    "base": 21,
    "d": 2,
}

small_loop = [
    ("norm", {}), 
    # ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold", {})
]

big_loop = [
    ("bdecomp", {"base": ops_params["base"]}), 
    ("norm", {}), 
    # ("batch", {}), 
    ("split", {"d": ops_params["d"]}), 
    ("fold", {})
]

# ops = 9 * loop + [("finish", {})]
ops = small_loop + big_loop + small_loop + big_loop + small_loop +  big_loop + [("finish", {})]

sim = Simulation(outer_ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show(brief=False)

print(f'VDF rounds T = {T}')


Ring parameters:
    conductor f: 24, degree phi: 8, modulus q: 2^62, beta_sis_2: 2^44
    SIS module rank n_sis: 153, target SIS security: 128, resulting SIS security: 128
    residue degree: 2, target Schwartz-Zippel security: 80, resulting Schwartz-Zippel security: 124
    |R_q| = 62.00 B, |R_q^(n_sis)| = 9.264 KB
 
Subtractive set parameters:
    cardinality: 3
    forward ell_2 expansion factor gamma_2: 2^0
    inverse ell_2 expansion factor theta_2: 2^3
    forward ell_inf expansion factor gamma_inf: 2^1
    inverse ell_inf expansion factor theta_inf: 2^3
 
Relation:
    H * F * W = Y
Statement:
    H: n_compress x (n_commit + n_rel)
    F: (n_commit + n_rel) x wdim
    Y: n_compress x rep
Witness:
    W: wdim x rep
    ||sigma(W)||_2 <= 2^log_beta_wit_2 measured in max column canonical ell_2-norm
    ||psi(W)||_inf <= 2^log_beta_wit_inf
Parameters:
    n_compress = 155, n_commit = 153, n_rel = 2
    wdim = 1222144, rep = 57
    log_2-norm (real | extr) = (14 |14 ), log_inf-norm 